In [ ]:
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

# In this demo d = 500 is for speed, the result is far away from the exact solution, at about 10000 it might be close.

d = 5 # degree of the polynomial, which is more or less the number of steps to approximate the imagary time evolution

def P(x):
    return jnp.cos(x)**(d-2)

# for performance they are moved into qsp_matrix
def R(theta):
    return jnp.array([
        [jnp.cos(theta/2), -jnp.sin(theta/2)],
        [jnp.sin(theta/2),  jnp.cos(theta/2)]
    ], dtype=jnp.complex128)

def U_signal(x, which):
    if which == 0:
        return jnp.array([[jnp.exp(1j*x), 0.0],
                          [0.0, 1.0]], dtype=jnp.complex128)
    else:
        return jnp.array([[1.0, 0.0],
                          [0.0, jnp.exp(-1j*x)]], dtype=jnp.complex128)

from jax import lax

@jax.jit
def qsp_matrix(angles, x):
    d = (angles.shape[0] - 1) // 2

    theta0 = angles[0]
    theta1 = angles[1:1+d]
    theta2 = angles[1+d:1+2*d]

    M = jnp.array([
        [jnp.cos(theta0/2), -jnp.sin(theta0/2)],
        [jnp.sin(theta0/2),  jnp.cos(theta0/2)]
    ], dtype=jnp.complex128)

    def loop0(M, theta):
        M = jnp.array([
        [jnp.cos(theta/2), -jnp.sin(theta/2)],
        [jnp.sin(theta/2),  jnp.cos(theta/2)]
    ], dtype=jnp.complex128) @ jnp.array([[jnp.exp(1j*x), 0.0],
                          [0.0, 1.0]], dtype=jnp.complex128) @ M
        return M, None

    def loop1(M, theta):
        M = jnp.array([
        [jnp.cos(theta/2), -jnp.sin(theta/2)],
        [jnp.sin(theta/2),  jnp.cos(theta/2)]
    ], dtype=jnp.complex128) @ jnp.array([[1.0, 0.0],
                          [0.0, jnp.exp(-1j*x)]], dtype=jnp.complex128) @ M
        return M, None

    M, _ = lax.scan(loop0, M, theta1)
    M, _ = lax.scan(loop1, M, theta2)

    return M

xs = jnp.linspace(0.0, 1.5, d * 23 // 11)
@jax.jit
def loss(angles):
    def loss_x(x):
        M = qsp_matrix(angles, x)
        amp = M[0,0]
        return jnp.abs(amp - P(x))**2

    return jnp.sum(jax.vmap(loss_x)(xs))

loss_grad = jax.jit(jax.grad(loss))


import numpy as np
from scipy.optimize import minimize

init_angles = np.random.uniform(0, np.pi, 2*d+1)

try:
    opt_angles = np.load(f'opt_angles_d{d}.npy')
    print(f"Loaded angles from opt_angles_d{d}.npy")
    print(f"Shape: {opt_angles.shape}")
except FileNotFoundError:
    print("No saved angles found, starting optimization.")
    res = minimize(loss, init_angles, method="BFGS", options={"maxiter":500}, jac=loss_grad)
    opt_angles = res.x
    print(res)
print("Optimized angles:", opt_angles)



No saved angles found, starting optimization.
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 1.2183514946096349e-06
        x: [ 4.564e+00  5.312e+00 ...  3.244e+00  1.729e+00]
      nit: 356
      jac: [-1.237e-06 -1.513e-06 ... -1.327e-06 -1.232e-06]
 hess_inv: [[ 4.230e+02 -1.033e+03 ... -1.381e+03  2.166e+03]
            [-1.033e+03  3.985e+03 ...  5.600e+03 -8.252e+03]
            ...
            [-1.381e+03  5.600e+03 ...  8.786e+03 -1.201e+04]
            [ 2.166e+03 -8.252e+03 ... -1.201e+04  1.761e+04]]
     nfev: 393
     njev: 393
Optimized angles: [ 4.56370415  5.31218822  1.74494391  3.05108012  1.49510123  2.64981753
  0.12721687 -0.50632195 -0.61128979  0.01034135 -0.240226    3.45210639
  0.27668614  0.89095981  0.32001958  0.30378006  2.48709783  4.84395648
  2.5320621   3.24415316  1.72913773]


In [ ]:
import pennylane as qml
import numpy as np
from scipy.sparse.linalg import eigsh

L_x = 2
L_y = 1
boundary_conditions = [False, False]

t = 1
UU = 8

# Parameters for Trotterization and filter alignment
optimal_x_for_filter = 0.02
n_steps = 1
order = 1

H_penny = qml.spin.fermi_hubbard('rectangle',[L_x, L_y], hopping=t, coulomb=UU, boundary_condition=boundary_conditions)

H_original = H_penny.sparse_matrix()

eig_vals_orig, eig_vecs_orig = eigsh(H_original, k=6, which='SA')
print(f"PennyLane Ground state energy: {np.min(eig_vals_orig)}")

H_matrix = H_original
eig_max_orig = eig_max = eigsh(H_matrix, k=1, which='LA', return_eigenvectors=False)[0]
eig_min = np.min(eig_vals_orig)
print(f"Max eigenvalue: {eig_max}")
print(f"Min eigenvalue: {eig_min}")

print("Shifting Hamiltonian...")
ops = H_penny.operands
coefs = [op.scalar for op in ops]
obs = [op.base for op in ops]
H_penny = qml.Hamiltonian(coefs, obs)

H_penny = (H_penny - qml.Identity(wires=H_penny.wires) * (eig_min - optimal_x_for_filter * (eig_max - eig_min))) / (eig_max - eig_min)
H_penny = qml.Hamiltonian(H_penny.coeffs, H_penny.ops)

H_matrix = H_penny.sparse_matrix()
eig_max_check = eigsh(H_matrix, k=1, which='LA', return_eigenvectors=False)[0]
eig_min_check = eigsh(H_matrix, k=1, which='SA', return_eigenvectors=False)[0]
print(f"Max eigenvalue: {eig_max_check}")
print(f"Min eigenvalue: {eig_min_check}")

ops = H_penny.operands
coefs = [op.scalar for op in ops]
print(f"Number of terms: {len(ops)}")
print(f"Coefficients: {coefs[:5]}...")  # Show first 5
print(f"Operators: {ops[:5]}...")  # Show first 5

obs = [op.base for op in ops]
grouped = qml.pauli.group_observables(obs, coefs, grouping_type='commuting')
print(f"Number of groups: {len(grouped)}")
print(f"Shape of first group: {len(grouped[0])}")
print(f"Shape of second group: {len(grouped[1])}")
for g in grouped[0]:
    print(len(g))
for g in grouped[1]:
    print(len(g))
    
obs_flat = [item for sublist in grouped[0] for item in sublist]
coeffs_flat = [item for sublist in grouped[1] for item in sublist]

grouped_H = qml.Hamiltonian(coeffs_flat, obs_flat) # , grouping_type='qwc') # the grouping here makes the lines above obsolete
eig_vals_grouped, _ = eigsh(grouped_H.sparse_matrix(), k=6, which='SA')
print(f"Check Ground state energy: {np.min(eig_vals_grouped)}")
print(f"shape of H {grouped_H.sparse_matrix().shape[0]}")

# Remap wires of grouped_H to make wire 0 free for control
def our_R(theta, wire):
    qml.RY(theta, wires=wire) # @ qml.RZ(np.pi, wires=wire)

wire_map = {i: i+1 for i in range(grouped_H.num_wires)}
grouped_H_new = qml.map_wires(grouped_H, wire_map)

def iterate(angles):
    d = len(angles)//2
    theta0 = angles[0]
    theta1 = angles[1:d+1]
    theta2 = angles[d+1:2*d+1]

    our_R(theta0, wire=0)
    
    # Erste Schleife: wirkt auf |0> Komponente
    for t in theta1:
        qml.ctrl(qml.TrotterProduct(grouped_H_new, time=1.0, n=n_steps, order=order, check_hermitian=True), control=0, control_values=0)
        our_R(t, wire=0)
        
    # Zweite Schleife: wirkt auf |1> Komponente
    for t in theta2:
        qml.ctrl(qml.TrotterProduct(grouped_H_new, time=-1.0, n=n_steps, order=order, check_hermitian=True), control=0, control_values=1)
        our_R(t, wire=0)

devtest = qml.device("lightning.qubit", wires=grouped_H.num_wires + 1)

@qml.qnode(devtest)
def my_circ_ctrl(angles=None):
    # Prepare some state
    for w in range(1, grouped_H.num_wires + 1):
        qml.Hadamard(w)
    iterate(angles)
    return qml.state()     

rr = my_circ_ctrl(opt_angles)

psi_0 = np.ones(2**grouped_H.num_wires) / np.sqrt(2**grouped_H.num_wires)
r_now = psi_0  # rr.reshape(-1,2)[:,0]
energy = np.vdot(r_now, H_penny.sparse_matrix() @ r_now).real / np.vdot(r_now, r_now).real
print(f"Expectation value of initial state: {energy}")
r_now = rr.reshape(2,-1)[0]
energy = np.vdot(r_now, H_penny.sparse_matrix() @ r_now).real / np.vdot(r_now, r_now).real
print(f"Expectation value of Hamiltonian: {energy}")
energy = np.vdot(r_now, H_original @ r_now).real / np.vdot(r_now, r_now).real
print(f"Expectation value of original Hamiltonian: {energy}")
print(f"Norm r_now: {np.vdot(r_now, r_now).real}, Norm rr: {np.vdot(rr, rr).real}")

PennyLane Ground state energy: -3.207750943219337
Max eigenvalue: 32.000000000000114
Min eigenvalue: -3.207750943219337
Shifting Hamiltonian...
Max eigenvalue: 1.0200000000000018
Min eigenvalue: 0.0199999999999995
Number of terms: 30
Coefficients: [np.float64(-0.014201418341272759), np.float64(-0.014201418341272759), np.float64(0.22722269346036414), np.float64(-0.014201418341272759), np.float64(-0.014201418341272759)]...
Operators: (-0.014201418341272759 * (Y(0) @ Z(1) @ Y(2)), -0.014201418341272759 * (X(0) @ Z(1) @ X(2)), 0.22722269346036414 * I([0, 1, 2, 3, 4, 5, 6, 7]), -0.014201418341272759 * (Y(1) @ Z(2) @ Y(3)), -0.014201418341272759 * (X(1) @ Z(2) @ X(3)))...
Number of groups: 2
Shape of first group: 3
Shape of second group: 3
8
14
8
8
14
8
Check Ground state energy: 0.019999999999999418
shape of H 256
Expectation value of initial state: 0.3383319196189042
Expectation value of Hamiltonian: 0.18613528154712922
Expectation value of original Hamiltonian: 2.6414986723736322
Norm r_n

In [ ]:
post_select_wire_1 = 0
post_select_wire_2 = grouped_H_new.num_wires + 3
work_wire_AA = grouped_H_new.num_wires + 1
ancilla_trotter_2 = grouped_H_new.num_wires + 2

dev = qml.device("lightning.qubit", wires=grouped_H_new.num_wires + 4)

def iterate(angles, good_wire=0):
    d = len(angles)//2
    theta0 = angles[0]
    theta1 = angles[1:d+1]
    theta2 = angles[d+1:2*d+1]

    our_R(theta0, wire=good_wire)
    
    # Erste Schleife: wirkt auf |0> Komponente
    for t in theta1:
        qml.ctrl(qml.TrotterProduct(grouped_H_new, time=1.0, n=n_steps, order=order, check_hermitian=True), control=good_wire, control_values=0)
        our_R(t, wire=good_wire)
        
    # Zweite Schleife: wirkt auf |1> Komponente
    for t in theta2:
        qml.ctrl(qml.TrotterProduct(grouped_H_new, time=-1.0, n=n_steps, order=order, check_hermitian=True), control=good_wire, control_values=1)
        our_R(t, wire=good_wire)

@qml.prod
def U2(wires):
    for wire in wires:
        qml.Hadamard(wires=wire)
    iterate(opt_angles)
    
@qml.prod
def oracle():
    qml.FlipSign(0, wires=0)

@qml.qnode(dev)
def full_result(iters, measure='X'):
    U2(wires=range(1, grouped_H_new.num_wires + 1))
    # qml.AmplitudeAmplification(U = U2(wires=range(1, grouped_H_new.num_wires + 1)),
    #                            O = oracle(),
    #                            iters = iters,
    #                            fixed_point=True,
    #                            work_wire=grouped_H_new.num_wires + 1
    #                            )
    ancilla = H_penny.num_wires + 2
    
    # # qml.measure(wires=0, postselect=0) # The good from amplitude amplification is 0
    # iterate(opt_angles, good_wire=H_penny.num_wires + 3)
    # # qml.measure(wires=0, postselect=0) # The good from iterate is set to H_penny.num_wires + 3
    
    qml.Hadamard(wires=ancilla)
    
    qml.ctrl(
        qml.TrotterProduct(grouped_H_new, time=1.0, n=1, order=order),
        control=ancilla
    )
    
    if measure == "X":
        qml.PauliX(ancilla)
    else:
        qml.PauliY(ancilla)
    # return qml.sample()
    return qml.state() # Here also samples could be returned (the good wires are 0 and H_penny.num_wires + 3)
    
    # if measure == "X":
    #     return qml.expval(qml.PauliX(ancilla))
    # else:
    #     return qml.expval(qml.PauliY(ancilla))
    
test_output_full = full_result(iters=10)


In [ ]:
np.linalg.norm(test_output_full), order

(np.float64(0.9999999999997828), 1)

In [ ]:
np.abs(test_output_full.reshape(2,2**grouped_H_new.num_wires, 2, 2, 2)**2)[0,:,:,:,0].sum()

np.float64(0.5068104684690369)

In [ ]:
post_selected_state = test_output_full.reshape(2,2**grouped_H_new.num_wires, 2, 2, 2)[0,:,:,:,0]
post_selected_state.shape

(256, 2, 2)

In [ ]:
import numpy as np

# Shape: (256, 2, 2)
state_tensor = post_selected_state  # already reshaped

# Trace über die letzten beiden Qubits: sum over axes -2, -1
# Wir erzeugen eine Density Matrix des Systems
rho_system = np.tensordot(state_tensor, np.conj(state_tensor), axes=([-2,-1], [-2,-1]))

# rho_system shape: (256, 256) → System-Density-Matrix
print("Shape rho_system:", rho_system.shape)

# Norm: Trace von rho_system
trace_rho = np.trace(rho_system).real
print("Norm nach Tracing (Trace):", trace_rho)


Shape rho_system: (256, 256)
Norm nach Tracing (Trace): 0.5068104684690369


In [ ]:
import numpy as np
import pennylane as qml

# rho_system: shape (256,256)
H_matrix = H_original

# Erwartungswert
energy = np.trace(rho_system @ H_matrix).real

# Norm / Postselection berücksichtigen
energy_normalized = energy / np.trace(rho_system).real

print("Postselection probability:", np.trace(rho_system).real)
print("Energy conditioned on postselection:", energy_normalized)


Postselection probability: 0.5068104684690369
Energy conditioned on postselection: 0.878611522931514


In [ ]:
from pennylane import transforms

decomposed_circuit = transforms.decompose(
    full_result,
    gate_set={"CNOT", "RX", "RY", "RZ", "H", "S", "T"}
)

specs = qml.specs(decomposed_circuit)(iters=1)
print(specs['resources'])

num_wires: 11
num_gates: 85782
depth: 57159
shots: Shots(total=None)
gate_types:
{'Hadamard': 8537, 'RY': 5290, 'RX': 41, 'RZ': 21730, 'CNOT': 33128, 'T': 17056}
gate_sizes:
{1: 52654, 2: 33128}


In [ ]:
output = qml.to_openqasm(decomposed_circuit)(iters=1)

In [ ]:
from qiskit import QuantumCircuit
qc = QuantumCircuit.from_qasm_str(output)

In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeProviderForBackendV2

provider = FakeProviderForBackendV2()

# Alle Backends
all_backends = provider.backends()
for b in all_backends:
    conf = b.configuration()
    print(f"Name: {b.name}, Qubits: {conf.num_qubits}, Basisgates: {conf.basis_gates}")

Name: fake_algiers, Qubits: 27, Basisgates: ['cx', 'id', 'rz', 'sx', 'x']
Name: fake_almaden, Qubits: 20, Basisgates: ['id', 'u1', 'u2', 'u3', 'cx']
Name: fake_armonk, Qubits: 1, Basisgates: ['id', 'rz', 'sx', 'x']
Name: fake_athens, Qubits: 5, Basisgates: ['id', 'rz', 'sx', 'x', 'cx', 'reset']
Name: fake_auckland, Qubits: 27, Basisgates: ['cx', 'id', 'rz', 'sx', 'x']
Name: fake_belem, Qubits: 5, Basisgates: ['id', 'rz', 'sx', 'x', 'cx', 'reset']
Name: fake_boeblingen, Qubits: 20, Basisgates: ['id', 'u1', 'u2', 'u3', 'cx']
Name: fake_bogota, Qubits: 5, Basisgates: ['id', 'rz', 'sx', 'x', 'cx', 'reset']
Name: fake_brisbane, Qubits: 127, Basisgates: ['ecr', 'id', 'rz', 'sx', 'x']
Name: fake_brooklyn, Qubits: 65, Basisgates: ['id', 'rz', 'sx', 'x', 'cx', 'reset']
Name: fake_burlington, Qubits: 5, Basisgates: ['id', 'u1', 'u2', 'u3', 'cx']
Name: fake_cairo, Qubits: 27, Basisgates: ['cx', 'ecr', 'id', 'rz', 'sx', 'x']
Name: fake_cambridge, Qubits: 28, Basisgates: ['id', 'u1', 'u2', 'u3', 'c

In [ ]:
from qiskit import transpile


backend = provider.backend("fake_boeblingen")

# Transpile your QC for this backend
qc_transpiled = transpile(qc, backend=backend, optimization_level=3)

print("Depth:", qc_transpiled.depth())
print("Gate counts:", qc_transpiled.count_ops())


Depth: 85388
Gate counts: OrderedDict({'cx': 64116, 'u1': 20327, 'u3': 16116, 'u2': 12912, 'measure': 11})


In [ ]:
from qiskit import transpile
from qiskit_aer import AerSimulator

# idealer Simulator
sim = AerSimulator()

# ggf. nochmal für Simulator transpilen
qc_sim = transpile(qc_transpiled, sim)

# Run
job = sim.run(qc_sim, shots=1024000)
result = job.result()
counts = result.get_counts()

print("Counts:", counts)


Counts: {'10110111111': 1, '01100000001': 1, '01101101010': 1, '11110001000': 1, '01010111111': 3, '00011001011': 1, '01110001000': 1, '01010000010': 1, '11011111110': 3, '11100100000': 3, '00111111100': 6, '10011101100': 10, '11100001001': 2, '00110111111': 2, '11011111100': 6, '11110000001': 1, '00111001111': 1, '01100100010': 3, '10011011101': 4, '00010011111': 12, '01101010110': 1, '10011001011': 3, '00111110111': 4, '11011110011': 8, '01101000000': 1, '00111110011': 5, '00110000100': 10, '10111111100': 6, '11101011000': 6, '01100000010': 3, '10001111011': 1, '11011001111': 5, '00010111101': 2, '10100111111': 8, '01011111011': 2, '10010111100': 3, '01011011111': 3, '01011110111': 3, '00000111111': 1, '00011111000': 10, '10011100111': 7, '00011100011': 1, '10110000100': 6, '01000000100': 33, '10011001101': 11, '10011111001': 17, '00110000010': 1, '11101100001': 6, '10000101111': 8, '01001000001': 2, '00000011111': 6, '10001110011': 10, '11110011001': 21, '00011001100': 15, '10100100

In [ ]:
sum(counts.values())

1024000

In [ ]:
# just test more or less the hamiltonian gate count


@qml.qnode(dev)
def test_result(iters, measure='X'):
    U2(wires=range(1, grouped_H_new.num_wires + 1))
    
    return qml.state()

from pennylane import transforms

decomposed_circuit = transforms.decompose(
    test_result,
    # gate_set={"CNOT", "RX", "RY", "RZ", "H", "S", "T"}
)

specs = qml.specs(decomposed_circuit)(iters=1)
print(specs['resources'])

output = qml.to_openqasm(decomposed_circuit)(iters=1)

from qiskit import QuantumCircuit
qc = QuantumCircuit.from_qasm_str(output)

from qiskit import transpile


backend = provider.backend("fake_boeblingen")

# Transpile your QC for this backend
qc_transpiled = transpile(qc, backend=backend, optimization_level=3)

print("Depth:", qc_transpiled.depth())
print("Gate counts:", qc_transpiled.count_ops())



num_wires: 9
num_gates: 4489
depth: 4401
shots: Shots(total=None)
gate_types:
{'Hadamard': 8, 'RY': 21, 'PauliX': 20, 'CH': 640, 'Toffoli': 2080, 'CRZ': 400, 'CRX': 640, 'PhaseShift': 40, 'RZ': 320, 'CNOT': 320}
gate_sizes:
{1: 409, 2: 2000, 3: 2080}
Depth: 41136
Gate counts: OrderedDict({'cx': 31552, 'u1': 11052, 'u3': 6729, 'u2': 5918, 'measure': 9})


In [ ]:
sum(specs['resources'].gate_types.values())

4489